In [ ]:
# Connect to MongoDB
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

with open('../data_base_key.txt', 'r') as file:
    file_content = file.read()

uri = file_content

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

## Read Data

In [28]:
db = client['db_products']
collection = db['products']

In [ ]:
for document in collection.find():
    print(document)

In [ ]:
collection.update_many({}, {'$rename': {'Produto': 'product',
                                        'Categoria do Produto': 'product category',
                                        'Preço': 'price',
                                        'Frete': 'shipping fee',
                                        'Data da Compra': 'purchase data',
                                        'Vendedor': 'seller',
                                        'Local da compra': 'place of purchase',
                                        'Avaliação da compra': 'purchase evaluation',
                                        'Tipo de pagamento': 'payment type',
                                        'Quantidade de parcelas': 'number of installments',
                                        'lat': 'latitude',
                                        'lon': 'longitude'}})

In [ ]:
collection.find_one()

In [ ]:
collection.distinct('product category')

In [39]:
query = {'product category': 'livros'}

book_list = []

for document in collection.find(query):
    book_list.append(document)

In [40]:
import pandas as pd

books_df = pd.DataFrame(book_list)

In [ ]:
collection.update_many({}, {'$rename': {'puchase data': 'purchase data'}})

In [ ]:
books_df['purchase data'] = pd.to_datetime(books_df['purchase data'], format='%d/%m/%Y')
books_df.info()

In [43]:
books_df['purchase data'] = books_df['purchase data'].dt.strftime('%Y-%m-%d')

In [44]:
books_df.to_csv('../data-processed/book_table.csv', index=False)

In [45]:
query = {'purchase data': {'$regex': '/202[1-9]'}}

products_list = []

for document in collection.find(query):
    products_list.append(document)


In [46]:
products_df = pd.DataFrame(products_list)

In [47]:
products_df['purchase data'] = pd.to_datetime(products_df['purchase data'], format='%d/%m/%Y')
products_df['purchase data'] = products_df['purchase data'].dt.strftime('%Y-%m-%d')

In [ ]:
products_df.head()

In [49]:
products_df.to_csv('../data-processed/products_sales_2021_present.csv')

In [50]:
client.close()